# Day 5: Binary Boarding

## Part 1

```
You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

Start by considering the whole range, rows 0 through 127.
F means to take the lower half, keeping rows 0 through 63.
B means to take the upper half, keeping rows 32 through 63.
F means to take the lower half, keeping rows 32 through 47.
B means to take the upper half, keeping rows 40 through 47.
B keeps rows 44 through 47.
F keeps rows 44 through 45.
The final F keeps the lower of the two, row 44.
The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

Start by considering the whole range, columns 0 through 7.
R means to take the upper half, keeping columns 4 through 7.
L means to take the lower half, keeping columns 4 through 5.
The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

BFFFBBFRRR: row 70, column 7, seat ID 567.
FFFBBBFRRR: row 14, column 7, seat ID 119.
BBFFBBFRLL: row 102, column 4, seat ID 820.
As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?
```

#### Part 1 - Solution Thoughts

This is basically converting FB -> 01 == binary row number, RL -> 10 == binary seat number

We're basically definining least and most significant digits from their description

SeatID = row * 8 + seat #

Quick method:
* Traverse list of tickets
    * Split each ticket into row & aisle
    * Convert to integer
    * Determine SeatID and set to True in array
    * Set row-seat to False
* Traverse seating chart
    * Find 'True' seat
    * Return row * 8 + aisle

In [ ]:
debug = True
FILENAME="data/Day_05_input.txt"

# Method 1

1. Initalize array of seats 0-127 x 0-7 -> 128x8 to O (Open)
1. For each ticket:
    1. Split the ticket into row & aisle
    1. Convert FB -> 01, convert to int for row
    1. Convert LR -> 01, convert to int for aisle
    1. Set the row/aisle element of the array to T (Taken)
    1. Store first and last rows filled for later use
1. For debugging - print out the array of seats
1. For each row with taken seats:
    1. Traverse the aisles
        1. If an "O" is found
            1. Show the row, aisle, and seatid

In [ ]:
import re

max_row = 64
min_row = 64
count = 0
seatarr = [['O' for x in range(8)] for x in range(128)] 

with open(FILENAME) as reader:
    for line in reader.readlines():
        line = line.strip()
        count += 1
        m = re.match('([FB]{7})([RL]{3})', line)
        row_b = re.sub('F','0',re.sub('B','1', m.group(1)))
        row = int(row_b ,2)
        aisle_b = re.sub('L','0',re.sub('R','1', m.group(2)))
        aisle = int(aisle_b,2)
        if row > max_row:
            max_row = row
        if row < min_row:
            min_row = row
        seatarr[row][aisle] = 'T'
        # print("{:-4}: {:8} -> {:7} {:3} => {:-4}{:-3} seatId: {}\tRows - min: {}\tmax: {}".format(count, line,row_b,aisle_b,row,aisle, seatid, min_row,max_row))

print("Rows -> Max: {}  Min: {}".format(max_row,min_row))
        
print("Row       Aisle")
print("     0 1 2 3 4 5 6 7")

for row in range(min_row, max_row + 1):
    print("{:-3}: {}".format(row, " ".join(seatarr[row])))
    for aisle in range(0,8):
        if seatarr[row][aisle] == 'O':
            ans_row = row
            ans_aisle = aisle
            

print("Row: ", ans_row, "\tAisle: ", ans_aisle, "\tSeatID: ", ans_row*8+ans_aisle)

# Method 2

Treat the full ticket as the seat identifier.
Put it into an array of seats.
Stash the lowest and highest possible seats.
Find the 0 inside the range of seats taken.


1. Intialize array from 0 to max seat id (128*8)
1. For each ticket
    1. Convert to binary
    1. Convert to int
    1. Set Array to 1 
    1. Store min and max seats based on rows taken
1. Traverse the seating array inside min & max range
    1. Find the seatid that is still 0

In [ ]:
import re
from pprint import pprint

max_seat = 444
min_seat = 444
seatarr = [0 for x in range(8*128)] 

with open(FILENAME) as reader:
    for line in reader.readlines():
        m = re.match('([FBRL]{10})',line.strip())
        seatid_b = re.sub('[FL]', '0', re.sub('[BR]', '1', m.group(1)))
        seatid = int(seatid_b, 2)
        seatarr[seatid] = 1
        if seatid > max_seat:
            max_seat = (seatid // 8)*8+7 # set max_seat to the end of this row
        if seatid < min_seat:
            min_seat = (seatid // 8)*8  # set the min_seat to the start of this row
            


for seat in range(min_seat, max_seat + 1):
    print("SeatID: {} => {}".format(seat, seatarr[seat]))
    if seatarr[seat] == 0:
        ans = seat
        
print("Available: ", ans)


### Method 3 - Math take 1 - split values

1. For each line of the file
    1. Parse the ticket into row and aisle
    1. Add the row*8 + aisle to the seat values (A)
    1. If a min/max store it
1. Sum the range from min to max (B)
2. Subtract seat values (A) from summed range (B)
    
    

In [ ]:
import re

max_row = 64
min_row = 64
seats = 0

with open(FILENAME) as reader:
    for line in reader.readlines():
        m = re.match(r'([FB]{7})([RL]{3})', line.strip())
        row_b = re.sub(r'[F]','0', re.sub(r'[B]', '1' , m.group(1)))
        row = int(row_b, 2)
        aisle_b = re.sub(r'[L]', '0', re.sub(r'[R]','1', m.group(2)))
        aisle = int(aisle_b, 2)
        if row > max_row:
            max_row = row
        if row < min_row:
            min_row = row
        seats += row*8 + aisle
        
all_seats = sum(range(min_row*8, (max_row+1)*8)) # could be done in O(1) with some math but lazy and this is clearer of what is happening
print("SeatID: ", all_seats - seats)

### Method 4 - Math take 2 - no splitting row/aisle

1. For each line of the file
    1. Convert the seat ID to an int based on the binary FL -> 0, BR -> 1
    1. Store as min/max if smaller/larger than previously seen extremes
    1. Add to total seats sum (A)
1. Sum seats from min to max (B)
1. Subtract total seats (A) from sum seats (B), this is your seatID


In [ ]:
import re


with open(FILENAME) as reader:
    for line in reader.readlines():
        line = line.strip()
        m = re.match('([FBRL]{10})', line)
        seatid = int( re.sub(r'[FL]','0', re.sub(r'[BR]','1', m.group(1) ) ), 2)
        seatids += seatid
        if seatid < min_seatid:
            min_seatid = (seatid // 8) * 8  # Get the first seat in this row, to avoid first seat edge case
        if  seatid > max_seatid:
            max_seatid = (seatid // 8) * 8 + 7  # Set to the last seat in the row, to avoid last seat edge case

all_seatids = sum(range(min_seatid, max_seatid + 1))
print("{} - {} = {}".format(all_seatids, seatids, all_seatids - seatids))
        


# Done !!!